In [265]:
from music21 import *
import pandas as pd
import os
import re
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
import lxml
import matplotlib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import linear_model
%matplotlib notebook

In [ ]:
configure.run()

In [266]:
import webbrowser
for motet in listOfMotets:
  superius = motet[0]
  lyrics = text.assembleLyrics(part)
  if 'exultavit' in lyrics:
    webbrowser.open('http://www.google.com/search?&q=' + lyrics)

('actNumber',
 'alternativeTitle',
 'associatedWork',
 'collectionDesignation',
 'commission',
 'composer',
 'copyright',
 'countryOfComposition',
 'date',
 'dedication',
 'groupTitle',
 'localeOfComposition',
 'movementName',
 'movementNumber',
 'number',
 'opusNumber',
 'parentTitle',
 'popularTitle',
 'sceneNumber',
 'textLanguage',
 'textOriginalLanguage',
 'title',
 'volume')

In [267]:
df_original = pd.read_csv('scores.csv')
df_original.Level.replace(['8+'], [9], inplace=True)
df_original['Level'] = pd.to_numeric(df_original['Level'])

df_original.info()
len(df_original)


TypeError: Cannot compare types 'ndarray(dtype=int64)' and 'str'

In [268]:
#let's split the Composer-Title column into Composer and Title
df_levels = df_original['Composer - Title'].str.split(':', n = 1, expand = True) 
# making separate Composer and Title columns:

df_original['Composer']= df_levels[0] 
df_original['Title']=df_levels[1]  
labels_to_drop = list(df_original.columns)[0:2]
print(labels_to_drop)
df = df_original.drop(columns=labels_to_drop)
df.head()



['Unnamed: 0', 'Composer - Title']


,Key,Type,Level,Composer,Title
0,F Major,Piece,9,Albéniz,Aragòn
1,D-flat Major,Piece,7,Albéniz,Barcarolle Catalene
2,A-flat Minor,Piece,9,Albéniz,La Vega
3,NaN,Piece,9,Albéniz,Rapsodia Espagñola
4,A Minor,Piece,8,Albéniz,Tango Espagnol


In [269]:
df.Composer.unique()
coreCorpus = corpus.corpora.CoreCorpus()
corpus_titles = coreCorpus.search('a', field='title' )
print(len(corpus_titles))


10723


In [270]:
corpusFilePaths = coreCorpus.getPaths()
#print(corpusFilePaths)
dirInfo = coreCorpus.directoryInformation
#first_composer = dirInfo[0].directoryTitle
list_composers = [dirInfo[i].directoryTitle for i in range(0,len(dirInfo))]
#print(list_composers)

#make a list of just last names: 
list_composers_lNames = [re.split('\s+',lName)[-1] for lName in list_composers]
print(list_composers_lNames)


['Airs', 'Bach', 'Beach', 'Beethoven', 'Chopin', 'Ciconia', 'Corelli', 'Bach', 'Files', 'Collection', 'Handel', 'Haydn', 'Joplin', 'Prez', 'demos', 'Luca', 'Folk', 'Monteverdi', 'Mozart', '(partial)', 'Collection', 'Palestrina', 'Collection', 'Schoenberg', 'Schubert', 'Schumann', 'Schumann', 'Exercises', 'Music', 'Verdi', 'Weber']


In [271]:
def findMatchingComposers(list1, list2):
    """go through list1 and isolate the composers who are also in list2"""
    listMatch = [composer for composer in list1 if composer in list2]
    return listMatch

#extract the common composers
comComposers = findMatchingComposers(list(df.Composer.unique()),list_composers_lNames)
print(comComposers)

['Bach', 'Beethoven', 'Chopin', 'Corelli', 'Handel', 'Haydn', 'Mozart', 'Schoenberg', 'Schubert', 'Schumann', 'Verdi', 'Weber']


In [295]:
#Extract features from files:
key = []
instrument = []
duration = []
initialTimeSig = []
changeTimeSig = []
highestOffset = []
highestTime = []
beatDuration = []
comp = []
title = []
iterator = 0
for composer in comComposers:
    pieces = corpus.getComposer(composer)
    #print(pieces)
    for piece in pieces:
        
        dur=''
        d_dig=''
        #print(piece)
        p = corpus.parse(piece)
        k = p.analyze('key') #returns the key
        key.append(k)
    
        chord = p.chordify() #returns the instrument and a stream object to extract other features
        
        d = chord.duration 
        #duration of piece
        dur = str(d)
        digits = r"\d+"
        d_dig = re.findall(digits,dur)
        d = float(d_dig[1])+float(d_dig[2])/(10*len(d_dig[2]))    
        print(d)
        duration.append(d)
        init_ts = p.recurse().getElementsByClass(meter.TimeSignature)[0]  #initial time signature
        it = str(init_ts)
        it_dig = re.findall(digits,it)
        init_ts = it_dig[1] + '/' + it_dig[2]
        print(init_ts)
        print(it_dig)
        initialTimeSig.append(init_ts)
        if len(set(p.recurse().getElementsByClass(meter.TimeSignature))):
            ts_change = 0
        else:
            ts_change = 1
        changeTimeSig.append(ts_change)    
        hOff = chord.highestOffset #highest offset
        highestOffset.append(hOff)
        hTime = chord.highestTime   #highest of all element offsets plus duration
        highestTime.append(hTime)
        #melInt = p.melodicIntervals()
        t = p.beatDuration
        beatDuration.append(t)
        instrs = chord.getInstruments()  #instrument part(s)
        isd = str(instrs)
        instrs = isd[21:-1]
        instrument.append(instrs)
        comp.append(composer)
        ti=os.path.basename(piece)
        title.append(ti)
        print(f'Composer: {composer}, title: {ti}')
        
#print(len(key))          
#print(len(instrument))

dictData = {'Composer': comp, 'Title': title, 'KeySignature': key, 'Instrument': instrument, 
                            'Duration':duration, 'initialTimeSignature':initialTimeSig, 'timeSignatureChange': changeTimeSig, 
                            'highestOffset':highestOffset, 'highestTimeD':highestTime, 'Tempo':beatDuration}
df_features = pd.DataFrame.from_dict(dictData)
df_features = df_features[['Composer','Title', 'KeySignature', 'Instrument', 
                            'Duration', 'initialTimeSignature', 'timeSignatureChange', 
                            'highestOffset', 'highestTimeD', 'Tempo']]

df_features.to_csv('features.csv')

80.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv1.6.mxl
88.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv10.7.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv101.7.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv102.7.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv103.6.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv104.6.mxl
52.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv108.6.mxl
66.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv11.6.mxl
44.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv110.7.mxl
80.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv111.6.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv112.5-sc.mxl
56.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv112.5.mxl
44.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv113.8.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv114.7.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv115.6.mxl
52.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv116.6.mxl
40.0
4/4


44.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.35-3c.mxl
159.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv248.42-4.mxl
84.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv248.42-s.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.46-5.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.5.mxl
32.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.53-5.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.59-6.mxl
272.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.64-6.mxl
64.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.64-s.mxl
61.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.9-1.mxl
32.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv248.9-s.mxl
52.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv25.6.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv250.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv251.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv252.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, titl

52.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv361.mxl
106.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv362.mxl
47.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv363.mxl
44.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv364.mxl
36.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv365.mxl
48.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv366.krn
72.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv366.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv367.mxl
97.0
3/4
['21', '3', '4']
Composer: Bach, title: bwv368.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv369.mxl
68.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv37.6.mxl
32.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv370.mxl
160.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv371.mxl
62.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv372.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv373.mxl
80.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv374.mxl
40.0
4/4
['21', '4', '4']
Composer: B

48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv90.5.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv91.6.mxl
76.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv92.9.mxl
60.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv93.7.mxl
64.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv94.8.mxl
60.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv95.7.mxl
40.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv96.6.mxl
48.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv97.9.mxl
56.0
4/4
['21', '4', '4']
Composer: Bach, title: bwv99.6.mxl
64.0
3/4
['21', '3', '4']
Composer: Bach, title: riemenschneider001.rntxt
53.0
4/4
['21', '4', '4']
Composer: Bach, title: riemenschneider002.rntxt
41.0
4/4
['21', '4', '4']
Composer: Bach, title: riemenschneider003.rntxt
41.0
4/4
['21', '4', '4']
Composer: Bach, title: riemenschneider004.rntxt
69.0
4/4
['21', '4', '4']
Composer: Bach, title: riemenschneider005.rntxt
33.0
4/4
['21', '4', '4']
Composer: Bach, title: riemenschneider006.rntxt
112.0
3

In [293]:
dictData = {'Composer': comp, 'Title': title, 'KeySignature': key, 'Instrument': instrument, 
                            'Duration':d, 'initialTimeSignature':initialTimeSig, 'timeSignatureChange': changeTimeSig, 
                            'highestOffset':highestOffset, 'highestTimeD':highestTime, 'Tempo':beatDuration}
df_features = pd.DataFrame.from_dict(dictData)
df_features = df_features[['Composer','Title', 'KeySignature', 'Instrument', 
                            'Duration', 'initialTimeSignature', 'timeSignatureChange', 
                            'highestOffset', 'highestTimeD', 'Tempo']]

df_features.to_csv('features.csv')